In [16]:
import numpy as np
import os

from PIL import Image

PATH = "../benchmarks_2/muc_scaling_id"
ASSUMPTION_SIGNATURE = "a"
CUSTOM_UNSAT_HEAD = ""

In [39]:
n_assumptions = 4000
n_mucs = 10

muc_size = 2800
density = (0.5 / (n_assumptions / (muc_size * 2)))
verbose = 0

mucs = []

while True:
    assignment = np.random.binomial(n=1, p=density, size=n_assumptions)
    candidate = {i + 1 for i, x in enumerate(assignment) if x == 1}
    if sum(assignment) != muc_size:
        continue

    if verbose:
        print(assignment, candidate)

    if any([candidate.issubset(muc) or muc.issubset(candidate) for muc in mucs]):
        if verbose:
            print("DENIED")
    else:
        if verbose:
            print("APPROVED")
        mucs.append(candidate)
        if len(mucs) >= n_mucs:
            break

In [40]:
print(len(mucs))
for muc in mucs:
    print(muc)

assumption_dir = f"{n_assumptions}_assumptions"
if PATH:
    assumption_dir = f"{PATH}/{assumption_dir}"
muc_dir = f"{assumption_dir}/{n_mucs}_mucs"
dirname = muc_dir

if not os.path.isdir(assumption_dir):
    os.mkdir(assumption_dir)
if not os.path.isdir(muc_dir):
    os.mkdir(muc_dir)


img_width = 1280
img_height = int((n_mucs / n_assumptions) * img_width)
img = np.zeros(shape=(n_mucs, n_assumptions), dtype=np.uint8) + 255
for m, muc in enumerate(mucs):
    for a in muc:
        img[m, a - 1] = 0
Image.fromarray(img).resize((img_width, img_height), resample=Image.BOX).save(f"{dirname}/visualization.jpeg")

with open(f"{dirname}/encoding.lp", "w") as f:
    f.write("% ---[ AUTO GENERATED MUC BENCHMARK ]\n")
    f.write(f"% ---[ assumptions : {n_assumptions} ]\n")
    f.write(f"% ---[ mucs : {n_mucs} ]\n\n")

    f.write(f"num(1..{n_assumptions}).\n")
    f.write("{" + ASSUMPTION_SIGNATURE + "(X): num(X)}.\n\n")

    for muc in mucs:
        f.write(f"{CUSTOM_UNSAT_HEAD}:- " + ", ".join([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) + ".\n")

with open(f"{dirname}/assumptions.lp", "w") as f:
    for a in range(n_assumptions):
        f.write(f"assume({ASSUMPTION_SIGNATURE}({a + 1})).\n")

with open(f"{dirname}/results.txt", "w") as f:
    results = dict()
    results["minimal"] = [list([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) for muc in mucs]

    minimum = [muc for muc in mucs if len(muc) == len(min(mucs, key=lambda x: len(x)))]
    results["minimum"] = [list([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) for muc in minimum]

    f.write(str(results).replace("'", '"'))

with open(f"{assumption_dir}/README.md", "w") as f:
    f.write(f"# Random MUC Benchmarks [{n_assumptions} Assumptions]\n\n")
    for s, subdir in enumerate(os.walk(assumption_dir)):
        if subdir[0] != assumption_dir:
            subdir_mucs = int(subdir[0].split("/")[-1].replace("_mucs", ""))
            f.write(f"## {s} : {subdir_mucs} MUCs\n\n")
            f.write(f"![]({subdir[0].split('/')[-1]}/visualization.jpeg)\n\n")

10
{2, 4, 6, 8, 11, 12, 13, 16, 17, 18, 19, 22, 23, 24, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 65, 66, 67, 68, 70, 71, 72, 73, 74, 76, 80, 81, 83, 84, 85, 86, 87, 88, 90, 92, 93, 94, 96, 98, 99, 101, 102, 103, 104, 105, 106, 107, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 124, 126, 127, 128, 131, 133, 134, 135, 136, 137, 140, 141, 142, 144, 145, 146, 148, 149, 150, 151, 154, 155, 156, 157, 158, 159, 161, 163, 166, 167, 168, 169, 172, 173, 174, 175, 179, 183, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195, 196, 197, 200, 201, 203, 204, 205, 206, 208, 209, 211, 212, 213, 215, 216, 217, 220, 221, 222, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 237, 238, 240, 241, 242, 244, 245, 246, 247, 248, 249, 253, 254, 256, 258, 259, 260, 261, 263, 264, 265, 266, 267, 268, 270, 271, 272, 273, 274, 275, 276, 277, 280, 281, 284, 285, 287, 288, 289, 290, 291, 292, 295, 296, 298, 299, 301, 302, 303, 304